# Huddle Test - Sensitivity Matrix

### Import Libraries

In [1]:
import os, sys
import shutil
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
from matplotlib.patches import Rectangle
from obspy import UTCDateTime, read, read_inventory
from obspy.clients.filesystem.sds import Client
from obspy.io.xseed import Parser
from obspy.signal import PPSD
from obspy.core.stream import Stream
from obspy.signal.util import next_pow_2
from obspy.signal.invsim import simulate_seismometer as seisSim
from obspy.core import AttribDict
from obspy.core.inventory import Inventory


### Define Functions 

In [2]:

def __calib_stream(st0, inv, path_to_xml, frange):  
    
    if path_to_xml[-1] == "/":
        path_to_xml = path_to_xml[:-1]
    
    st = st0.copy() # work on copy
    
    st.detrend('demean') # needed because submitted data may be sliced from longer (demeaned) original
    
    # remove poles and zeros ## but NOT sensitivity ## and replace data with its spectrum
    NPTS = max([tr.stats.npts for tr in st])

    nfft = next_pow_2(NPTS)

    freqs = np.fft.fftfreq(nfft, st[0].stats.delta)[:int(nfft/2+1)] # only real frequencies
    
    freqs[-1] = np.abs(freqs[-1]) # max frequency should be positive

    for tr in st:
        
        ## read local response file 
        inv = read_inventory(path_to_xml + "/" + tr.stats.station + ".xml") #response files
        
        ## extract poles and zeros from response file
        r = inv[0][0][0].response.get_paz()
        
        paz = {'poles': r.poles, 'zeros': r.zeros, 'gain': r.normalization_factor}
        
        ## simulate response function
        tr.simulate(paz_remove=paz, remove_sensitivity=False)
        
        ## transform back to time domain
        tr.data = np.fft.rfft(tr.data, nfft)
        
        print(tr.stats.station + ' response removed')

#     return calib_spec(st, freqs, frange=frange)
    return st, freqs, frange


In [3]:

def __calib_spec(sp, freqs, frange):  #enter frequency range
    '''
    calibrate relative sensitivity matrix of __SPECTRA__ in Stream object sp

    sp:     Stream, contains rfft spectra of traces to be calibrated against each other
    freqs:  np.array, frequencies corresponding to samples in Spectra sp

    frange: list, frequency range in which the amplitude response of tr and trref will be compared, 
            commonly around calibration frequency, which is often 1Hz. Thus default is 3s to 3Hz

    returns sensitivity matrix
            each row is a reference instrument, each column abs(XCORR/ACORR), averaged over frequencies in frange
            sensitivity is thus in each row RELATIVE to the reference instrument, i.e. diagonal of matrix = 1
    '''

    sens = np.zeros((sp.count(), sp.count()))
    for i, tr in enumerate(sp):
        acorr = tr.data * np.conj(tr.data) # is real, np.all(acorr.imag == 0) is True
        for j in range(i, sp.count()):
            xcorr = sp[j].data * np.conj(tr.data)
            sens[i, j] = np.abs((xcorr/acorr)[np.where(np.logical_and(freqs>min(frange), freqs<max(frange)))]).mean()
#           if i != j: sens[j, i] = 1 / sens[i, j] ## fill lower triangle of matrix
            if i != j: sens[j, i] = np.nan
        
    return sens


In [4]:

def __load_streams(sensors, s0, s1, workdir, chn):

    ## make list with yeardoy strings for loading the data
    days = []
    for i in range(s0.julday, s1.julday+1):
        if i < 10:
            i = f"00{i}"
        elif (i >= 10 and i < 100):
            i = f"0{i}"
        days.append(f'{s0.year}{i}')
    
    
    if workdir[-1] == "/":
        workdir = workdir[:-1]
    
    
    st_all_sensor_all_days = Stream()

    
    for sensor in sensors:

        net, sta, loc, cha = sensor.split(".")
        st_one_sensor_all_days = Stream()

        cha = chn
        for day in days:

            year = day[:4]
            doy = day[4:]
            st_tmp = read(f"{workdir}/{year}/{net}/{sta}/{cha}.D/{net}.{sta}.{loc}.{cha}.D.{year}.{doy}")

            st_one_sensor_all_days += st_tmp

        st_one_sensor_all_days.merge()       
        print(st_one_sensor_all_days)
        st_all_sensor_all_days += st_one_sensor_all_days
        
        st_all_sensor_all_days.trim(s0, s1)
        
    return st_all_sensor_all_days


In [37]:

def __plot_matrix(pct, title_str, lbls, fname=None, cmap=plt.get_cmap(), full=False, 
                      clabel='deviation in spectral amplitude [%]', vmax=None):
    '''
    plot calibration matrix

    pct: np.array, 2D matrix of relative sensitivities (result from calib_stream)
    title_str: str, title of figure
    lbls: str, x- and y-axis labels

    optional:
        fname: str, filename of figure to be saved, if 'None' figure will be displayed
        cmap: matplotlib.cmap
        full: bool, fill up half matrix with inverse values, default: False
        clabel: str, colorbar label, what is shown in matrix pct?
        vmax: float or None, limit of colorbar or None (default) which determines it automatically from standard deviation
    '''
    

    def __myfmt(a):
            if np.log10(abs(a)) > 3: return '%.0e' % a
            elif np.log10(abs(a)) >= 0 or np.log10(abs(a)) == -np.inf: return '%i' % a # a>=1 or a=0
            elif np.log10(abs(a)) >= -1: return '%.1f' % a 
            elif np.log10(abs(a)) >= -2: return '%.2f' % a
            else: return '%.0e' % a


    font = 15
    
    flip = 0
    
    if full:
        m = pct.copy()
        m[np.isnan(m)] = 0.
        n = np.triu(1/m,1).T
        n[np.isnan(n)] = 0.
        pct = n+m

    if np.isnan(pct[-1, 0]) or full: 
        pct = np.flipud(pct) # flip to lower triangle
        flip = 1

    plt.figure(figsize=(15,10))
    
    if vmax is None: vmax = min(np.nanstd(pct), 30) # cap to 30%
    plt.imshow(pct, origin="upper", interpolation="nearest", cmap=cmap, vmin=-vmax, vmax=vmax)
    cb = plt.colorbar(shrink=.8)
    cb.set_label(clabel, fontsize=font-1)
    
    plt.title(title_str, fontsize=font)
    plt.xlabel('Instrument ID', fontsize=font)
    plt.ylabel('Reference Instrument ID', fontsize=font)

    plt.xticks(range(len(lbls)), lbls, rotation=75, ha='center', fontsize=font-2)
    plt.yticks(range(len(lbls)), lbls[::-1] if flip else lbls, fontsize=font-2)

    ## overlay values
    i = np.arange(len(lbls))
    x, y = np.meshgrid(i, i)
    for k, l in zip(x.flatten(), y.flatten()):
        if not np.isnan(pct[l, k]): 
            plt.text(k, l, __myfmt(pct[l, k]), fontsize=font-2, ha='center', color='w', 
                     bbox={'pad': 2, 'alpha': .07, 'color': 'k', 'edgecolor': "none"}) 

    ## save or show
    if fname is not None: 
        plt.savefig(fname,
                    dpi=300,
                    bbox_inches="tight",
                    pad_inches=0.1,
                   )
        plt.close()
    else:
        plt.show()

    return


In [6]:

def __get_gain(path, xml_file):

    if path[-1] == "/":
        path = path[:-1]
    
    ## access response information
    r = read_inventory(path+'/'+xml_file)[0][0][2].response
    

    ## split it up to get gain value (could surely be done more elegant...)
#     gain = float(str(r).split("\n")[4].split(" ")[8]) * float(str(r).split("\n")[5].split(" ")[8]) 
    
    gain = 1
    for i in range(4, len(str(r).split("\n"))):
        gain *= float(str(r).split("\n")[i].split(" ")[8])

    return float(gain)


### Configurations

In [34]:
## set paths to data and plots
##
## assumed file structure: path - year - network - stations - components - BW.29422..BHE.D.2021.041
## (e.g.  path - 2021 - BW - 29422 - HHE.D - BW.29422..BHE.D.2021.041)

path = '/home/andbro/Desktop/Huddle_Test/Huddle_Test_1'
path2plot = '/home/andbro/Desktop/Huddle_Test/Huddle_Test_1/plots'


## time window to analyze data
s0 = UTCDateTime('2021-02-11T18:00:00')
s1 = UTCDateTime('2021-02-12T12:00:00')
# s1 = UTCDateTime('2021-02-11T09:59:59')

## choose colormap
cmap = cm.seismic

## change title string
title_str = 'Huddle Test 1'

##data client
client = Client(sds_root=path)

## set channels
channels = ["HHZ", "HHN", "HHE"]

## stations to be included 
seed_id_1 = "BW.29422..HH*"
seed_id_2 = "BW.59811..HH*"
seed_id_3 = "BW.49507..HH*"
seed_id_4 = "BW.horizon..HH*"

sensors = [seed_id_1, seed_id_2, seed_id_3, seed_id_4]


## frequency band
f_lower, f_upper = 0.01, 1.0


## Save figure if True, else display it
save = True


In [35]:
# ## set paths to data and plots
# ##
# ## assumed file structure: path - year - network - stations - components - BW.29422..BHE.D.2021.041
# ## (e.g.  path - 2021 - BW - 29422 - HHE.D - BW.29422..BHE.D.2021.041)

# path = '/home/andbro/Desktop/Huddle_Test/Huddle_Test_2'
# path2plot = '/home/andbro/Desktop/Huddle_Test/Huddle_Test_2/plots'


# ## time window to analyze data
# s0 = UTCDateTime('2021-02-12T21:00:00')
# s1 = UTCDateTime('2021-02-14T23:00:00')
# # s1 = UTCDateTime('2021-02-11T09:59:59')

# ## choose colormap
# cmap = cm.seismic

# ## change title string
# title_str = 'Huddle Test 2'

# ##data client
# client = Client(sds_root=path)

# ## set channels
# channels = ["HHZ", "HHN", "HHE"]

# ## stations to be included 
# seed_id_1 = "BW.29422..HH*"
# seed_id_2 = "BW.49510..HH*"
# seed_id_3 = "BW.109610..HH*"
# seed_id_4 = "BW.horizon..HH*"

# sensors = [seed_id_1, seed_id_2, seed_id_3, seed_id_4]


# ## frequency band
# f_lower, f_upper = 0.01, 1.0

# ## Save figure if True, else display it
# save = True


### Calculations and Plotting

In [39]:

## ___________________________
## LOOP over 1hr data windows

for chn in channels:
    print('Processing channel: ' + chn)
    
    cnt = 0 # count number of spectral ratios in sum
    
    timestr = "%s - %s" % (s0.strftime("%Y.%m.%dT%H:%M"), (s1-1).strftime("%Y.%m.%dT%H:%M"))
    
    
    ## load one channel for all sensors in one stream
    print("\nreading streams...")
    st1 = __load_streams(sensors, s0, s1, path, chn)

#     st1.detrend('demean')
#     st1.filter('lowpass', freq=1.0, corners=4, zerophase=True)
    

    Z = st1.copy()
    Z.merge()
    Z.sort()
    print(Z.__str__(extended=True))

    if save:
        fout = path2plot + f"/{chn}_traces.png"
        Z.plot(outfile=fout)
    else:
        Z.plot();
    
    
    t0, t1 = s0, s1

    while t0 <= t1 - 3600:
        print("\nprocessing t0: %s" % str(t0))
        
        ## call calibration function
        st, freqs, frange = __calib_stream(Z.slice(starttime=t0, endtime=t0+3600), 1, path, [f_lower, f_upper])
        
        ## call calibration_spec
        sens = __calib_spec(st, freqs, frange)
        
        if cnt == 0:
            senssum = sens
        else:
            senssum = np.dstack((senssum, sens))
        cnt += 1
        t0 += 3600
    
    ## ___________________________
    ## Postprocessing
    
    lbls = [tr.stats.station for tr in Z]
#     print(lbls)
    
    m = np.median(senssum, axis = 2 ) # FINAL sensitivity matrix is median of all calculated matrices
    
    pct = (m - 1) * 100 # deviation in percent

    d = m.copy()
    

    
    
    for i, sta1 in enumerate(lbls):
        gain1 =  __get_gain(path, f'{sta1}.xml')
        
        
        for j, sta2 in enumerate(lbls):
            gain2 =  __get_gain(path, f'{sta2}.xml')


            d[i,j] = (m[i,j]*gain1 - gain2) * 100 / gain2 # relative deviation in % from nominal value
            print(d[i,j])
        
    ## _________________________
    ## Plotting
            
    fname = f'{path2plot}/huddle_result_{chn}_RR_{f_lower}-{f_upper}Hz.png'  if save else None
    
    __plot_matrix(d, title_str + ' %s - %s' % (timestr, chn), 
                      lbls, 
                      fname=fname, 
                      cmap=cmap, 
                      clabel='deviation from nominal sensitivity in %', 
                      vmax=3,
                     )
    
    if fname is not None:
        np.savez(fname.rsplit(".", 1)[0], senssum=senssum, lbls=lbls)

        
print("\n DONE")

Processing channel: HHZ

reading streams...
1 Trace(s) in Stream:
BW.29422..HHZ | 2021-02-11T00:00:00.000000Z - 2021-02-12T13:34:10.845000Z | 200.0 Hz, 27050170 samples
1 Trace(s) in Stream:
BW.59811..HHZ | 2021-02-11T00:00:00.000000Z - 2021-02-12T13:12:15.950000Z | 200.0 Hz, 26787191 samples
1 Trace(s) in Stream:
BW.49507..HHZ | 2021-02-11T00:00:00.000000Z - 2021-02-12T13:12:15.950000Z | 200.0 Hz, 26787191 samples
1 Trace(s) in Stream:
BW.horiz..HHZ | 2021-02-11T00:00:00.000000Z - 2021-02-12T23:59:59.995000Z | 200.0 Hz, 34560000 samples
4 Trace(s) in Stream:
BW.29422..HHZ | 2021-02-11T18:00:00.000000Z - 2021-02-12T12:00:00.000000Z | 200.0 Hz, 12960001 samples
BW.49507..HHZ | 2021-02-11T18:00:00.000000Z - 2021-02-12T12:00:00.000000Z | 200.0 Hz, 12960001 samples
BW.59811..HHZ | 2021-02-11T18:00:00.000000Z - 2021-02-12T12:00:00.000000Z | 200.0 Hz, 12960001 samples
BW.horiz..HHZ | 2021-02-11T18:00:00.000000Z - 2021-02-12T12:00:00.000000Z | 200.0 Hz, 12960001 samples

processing t0: 2021-0

/home/andbro/anaconda3/envs/obs/lib/python3.7/site-packages/obspy/core/inventory/response.py:1895: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


horiz response removed

processing t0: 2021-02-11T19:00:00.000000Z
29422 response removed
49507 response removed
59811 response removed


/home/andbro/anaconda3/envs/obs/lib/python3.7/site-packages/obspy/core/inventory/response.py:1895: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


horiz response removed

processing t0: 2021-02-11T20:00:00.000000Z
29422 response removed
49507 response removed
59811 response removed


...

horiz response removed

processing t0: 2021-02-12T11:00:00.000000Z
29422 response removed
49507 response removed
59811 response removed


/home/andbro/anaconda3/envs/obs/lib/python3.7/site-packages/obspy/core/inventory/response.py:1895: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


horiz response removed
0.0
-0.139971662299
0.239562871104
-1.89188480705
nan
0.0
1.13430542812
0.1729073029
nan
nan
0.0
0.222337235819
nan
nan
nan
0.0


/home/andbro/anaconda3/envs/obs/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10
/home/andbro/anaconda3/envs/obs/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log10
/home/andbro/anaconda3/envs/obs/lib/python3.7/site-packages/ipykernel_launcher.py:62: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.



 DONE
